# Basic BoW model 

In [1]:
from collections import defaultdict
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [2]:
def cleanup_line(line):
    # removes punctuation
    line = re.sub(r'[.,"!#/]+', ' ', line, flags=re.MULTILINE)
    # removs \\n
    line = re.sub(r'\\n', ' ', line, flags=re.MULTILINE)
    # removes retweets (e.g. @FER)
    line = re.sub(r'@\w+', '', line, flags=re.MULTILINE)

    return line

In [3]:
def vectorize_X(tweet, vocab):
    # obsolete
    # tweet = re.sub(r'[.,"!#]+', ' ', tweet, flags=re.MULTILINE)
    #tweet = re.sub(r'@\w+', ' ', tweet, flags=re.MULTILINE)

    # splits tags into 2, e.g. #happy -> #, happy
    # TODO: tags could be useful for classification
    tweet_toks = word_tokenize(tweet)
    X_ = []
    for w in vocab:
        X_.append(1 if w in tweet_toks else 0)

    return X_

In [4]:
def create_bow(input_file):
    vocab = set()
    X, y = [], []
    stop_words = set(stopwords.words('english'))
    with open(f'data/{input_file}') as fin:
        stemmer = PorterStemmer()
        #stemmer = SnowballStemmer()

        bag = defaultdict(int)
        cols = fin.readline().split()
        for line in fin:
            fields = cleanup_line(line).lower().strip().split('\t')
            # for w in fields[1].split():
            for w in word_tokenize(fields[1]):
                if w.strip() in stop_words:
                    continue
                vocab.add(w.strip())
                bag[stemmer.stem(w.strip())] += 1

        # go back to the 1st example
        fin.seek(1, 0)
        for line in fin:
            fields = cleanup_line(line).lower().strip().split('\t')
            X.append(vectorize_X(fields[1], vocab))
            y.append(fields[2:])

    return X, y, sorted(vocab), bag

In [5]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [6]:
X, y, vocab, bow = create_bow('2018-E-c-En-dev.txt')
v = vectorize_X("hypo hypo throw ahu", vocab)

#OneVsRestClassifier(LinearSVC(random_state=42)).fit(X, y).predict(X)